## USE pretrained network to output keypoint's description

In [1]:
from __future__ import division, print_function
import glob
import os
import cv2
import PIL
import random
import numpy as np
# import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import torch
import torch.nn.init
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torch.nn.functional as F
import torchvision.datasets as dset
import torchvision.transforms as transforms
from tqdm import tqdm
from torch.autograd import Variable
from copy import deepcopy, copy
from config_profile import args
from Utils import cv2_scale36, cv2_scale, np_reshape, np_reshape64
from munkres import Munkres

from descriptor_CNN3 import DesNet
model = DesNet()
if args.cuda:
    model.cuda()
weight_path = "checkpoint_b.pth"
trained_weight = torch.load(weight_path)
model.load_state_dict(trained_weight['state_dict'])

# load patches
patches_a_dir = "../keypoint_detector/patches-a.pt"
patches_i_dir = "../keypoint_detector/patches-i.pt"
patches_q_dir = "../keypoint_detector/patches-q.pt"
patches_a = torch.load(patches_a_dir)
patches_i = torch.load(patches_i_dir)
patches_q = torch.load(patches_q_dir)

print(patches_a.shape)
print(patches_i.shape)
print(patches_q.shape)
patches_a =  patches_a.view(-1, 1, 32, 32).cuda()
patches_i =  patches_i.view(-1, 1, 32, 32).cuda()
patches_q =  patches_q.view(-1, 1, 32, 32).cuda()

#description = model(patches)
with torch.no_grad():
    description_a = model(patches_a)
    description_i = model(patches_i)
    description_q = model(patches_q)

description_a = description_a.view(-1, 30, 128)
description_i = description_i.view(-1, 30, 128)
description_q = description_q.view(-1, 30, 128)
print(description_a.shape)
print(description_i.shape)
print(description_q.shape)

print("Done!")

torch.Size([175, 30, 1, 32, 32])
torch.Size([140, 30, 1, 32, 32])
torch.Size([35, 30, 1, 32, 32])
torch.Size([175, 30, 128])
torch.Size([140, 30, 128])
torch.Size([35, 30, 128])
Done!


In [7]:
similarity_matrix = np.zeros((35,140))
#print(description_q[0].shape)
#print(description_i[0].shape)
hungarian = Munkres()
for i in range(0,35):
    top = 0
    for j in range(0,140):
        costs = np.zeros((30,30))
        for k in range(0,30):
            for l in range(0,30):
                costs[k][l] = np.linalg.norm(np.subtract(description_q[i][k].cpu().numpy(), description_i[j][l].cpu().numpy()),ord=2)
        test = np.subtract(description_q[i].cpu().numpy(), description_i[j].cpu().numpy())
        cost_indecies = hungarian.compute(np.copy(costs))
        tmp = np.zeros(len(cost_indecies))
        for k in range(0,len(cost_indecies)):
            tmp[k] = np.exp(-(costs[cost_indecies[k][0]][cost_indecies[k][1]]))
        similarity_matrix[i][j]=tmp.sum()
        if(top < similarity_matrix[i][j]):
            top = similarity_matrix[i][j]
            print((i,j))
        #print(similarity_matrix[i][j])
print(similarity_matrix)
# Check point: saving the tensor to prevent re computing it
print("Saving a checkpoint...")
torch.save(similarity_matrix, "checkpoint-1-to-1.pt")
print("Done!")


(0, 0)


KeyboardInterrupt: 

In [8]:
# start from here to reload the tensor
similarity_matrix = torch.load("checkpoint-1-to-1.pt")
print(similarity_matrix.shape)
print(similarity_matrix)
print("Done!")

(35, 140)
[[14.78335888 14.76507671 13.94114402 ...  8.35769791  8.49947696
   8.39132851]
 [ 8.61657291  8.65183173  8.62217608 ...  8.55982659  8.76162621
   8.38833227]
 [ 8.92609557  9.05985828  8.76591278 ...  8.24642189  8.18303793
   8.48164175]
 ...
 [ 8.19770111  8.37336017  8.1499579  ...  8.31396344  8.3035486
   8.6731017 ]
 [ 8.03349518  8.28073153  7.94179169 ...  7.9018665   7.83969328
   8.14647369]
 [ 8.61732855  8.49051534  8.56383987 ...  9.6003048  16.68723528
  11.0266813 ]]
Done!


In [ ]:
similarity_matrix = np.zeros((35,140))
#print(description_q[0].shape)
#print(description_i[0].shape)
hungarian = Munkres()
for i in range(0,35):
    top = 0
    for j in range(0,140):
        costs = np.zeros((30,30))
        for k in range(0,30):
            for l in range(0,30):
                costs[k][l] = np.linalg.norm(np.subtract(description_q[i][k].cpu().numpy(), description_i[j][l].cpu().numpy()),ord=2)
        test = np.subtract(description_q[i].cpu().numpy(), description_i[j].cpu().numpy())
        cost_indecies = hungarian.compute(np.copy(costs))
        tmp = np.zeros(len(cost_indecies))
        for k in range(0,len(cost_indecies)):
            tmp[k] = np.exp(-(costs[cost_indecies[k][0]][cost_indecies[k][1]]))
        similarity_matrix[i][j]=tmp.sum()
        if(top < similarity_matrix[i][j]):
            top = similarity_matrix[i][j]
            print((i,j))
        #print(similarity_matrix[i][j])
print(similarity_matrix)
print("Done!")

In [ ]:
for i in range(0,35):
    top = 0
    for j in range(0,140):
        costs = np.zeros((30,30))
        for k in range(0,30):
            for l in range(0,30):
                costs[k][l] = np.linalg.norm(np.subtract(description_q[i][k].cpu().numpy(), description_i[j][l].cpu().numpy()),ord=2)
        test = np.subtract(description_q[i].cpu().numpy(), description_i[j].cpu().numpy())
        tmp = np.zeros(costs.shape)
        for k in range(0,len(cost_indecies)):
            tmp[k] = np.exp(-(costs.sum))
        similarity_matrix[i][j]=tmp.sum()